## clean_expenditures_data
1. process fiscal year expenditure csv files
2. extract monthly expenditure by animal services department
3. save data as a table in aws postgres database

### Data Source:  https://data.louisvilleky.gov/dataset/expenditures-data

In [1]:
import pandas as pd

animal_expenditures = []
#animal_expenditures.columns =['year','month','expenses']

#2008-2012 work
years = [i for i in range(2008,2012)]

for year in years:
    filename='/home/jieliang/proj3/submission/data/Expenditures_FiscalYear'+str(year)+'.csv'
    df = pd.DataFrame()
    df = pd.read_csv(filename)
    
    mask = df['Sub_Agency_Name']=='Metro Animal Services' #or df['Agency_Name']=='Metro Animal Services'
    df = df[mask]
    df.InvoiceDt=df.InvoiceDt.apply(pd.to_datetime)
    df['month'] = df.InvoiceDt.dt.month
    df['year'] = df.InvoiceDt.dt.year
    
    df.InvoiceAmt = df['InvoiceAmt'].astype(float)
    df['year_month'] = df.apply(lambda row: str(row.year)+'-' + str(row.month), axis=1)
    #df = df['year','month','InvoiceAmt','year_month']

    animal_expenditures.append(df)

/home/jieliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jieliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (8,13,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 857 entries, 74938 to 75794
Data columns (total 23 columns):
Fiscal_Year           857 non-null int64
Budget_Type           857 non-null object
Agency_Name           857 non-null object
Sub_Agency_Name       857 non-null object
DepartmentName        857 non-null object
Sub_DepartmentName    857 non-null object
Category              857 non-null object
Sub_Category          857 non-null object
Stimulus_Type         0 non-null object
Funding_Source        857 non-null object
Vendor_Name           857 non-null object
InvoiceID             857 non-null int64
InvoiceDt             857 non-null datetime64[ns]
InvoiceAmt            857 non-null float64
DistributionAmt       857 non-null float64
CheckID               857 non-null int64
CheckDt               857 non-null object
CheckAmt              857 non-null object
CheckVoidDt           857 non-null object
Unnamed: 19           0 non-null float64
month                 857 non-null int64
year

In [3]:
len(animal_expenditures)

4

In [4]:
years = [i for i in range(2012,2019)]

for year in years:
    filename='/home/jieliang/proj3/submission/data/Expenditures_FiscalYear'+str(year)+'.csv'
    df = pd.DataFrame()
    df = pd.read_csv(filename)
    
    mask = df['Agency_Name']=='Metro Animal Services' #or df['Agency_Name']=='Metro Animal Services'
    df = df[mask]
    df.InvoiceDt=df.InvoiceDt.apply(pd.to_datetime)
    df['month'] = df.InvoiceDt.dt.month
    df['year'] = df.InvoiceDt.dt.year
    
    df.InvoiceAmt = df['InvoiceAmt'].astype(float)
    df['year_month'] = df.apply(lambda row: str(row.year)+'-' + str(row.month), axis=1)
    #df = df['year','month','InvoiceAmt','year_month']

    animal_expenditures.append(df)

/home/jieliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jieliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,12,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/jieliang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,4,5,6,7,9,10,12,16,18) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
animal_expenditures[7].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1478 entries, 35 to 70201
Data columns (total 22 columns):
Fiscal_Year           1478 non-null float64
Budget_Type           1478 non-null object
Agency_Name           1478 non-null object
Sub_Agency_Name       1478 non-null object
DepartmentName        1474 non-null object
Sub_DepartmentName    0 non-null object
Category              1478 non-null object
Sub_Category          1478 non-null object
Stimulus_Type         0 non-null float64
Funding_Source        1478 non-null object
Vendor_Name           1478 non-null object
InvoiceID             1478 non-null float64
InvoiceDt             1478 non-null datetime64[ns]
InvoiceAmt            1478 non-null float64
DistributionAmt       1478 non-null float64
CheckID               1478 non-null float64
CheckDt               1478 non-null object
CheckAmt              1478 non-null float64
CheckVoidDt           1478 non-null object
month                 1478 non-null int64
year                  1

In [6]:
animal_expenditures[7].groupby('year_month')['InvoiceAmt'].sum().index

Index(['2013-10', '2013-12', '2014-1', '2014-10', '2014-11', '2014-12',
       '2014-2', '2014-3', '2014-4', '2014-5', '2014-6', '2014-7', '2014-8',
       '2014-9', '2015-1', '2015-2', '2015-3', '2015-4', '2015-5', '2015-6',
       '2015-7'],
      dtype='object', name='year_month')

In [7]:
expenditures_year_month = pd.Series()

In [8]:
for animal_expenditure in animal_expenditures:
    expenditures_year_month = expenditures_year_month.append(animal_expenditure.groupby('year_month')['InvoiceAmt'].sum())

In [9]:
expenditures_year_month

2006-11      9066.95
2006-9       2085.21
2007-10     49057.56
2007-11     39262.93
2007-12     48468.56
2007-3        371.22
2007-4        255.65
2007-5      14457.69
2007-6      22044.06
2007-7      42975.42
2007-8      42867.97
2007-9      66793.83
2008-1      43369.15
2008-2      47588.89
2008-3      39477.55
2008-4      34348.57
2008-5      54367.48
2008-6      55511.81
2008-7       2667.18
2007-10       230.58
2007-8        594.81
2008-10     40089.66
2008-11     30462.52
2008-12     39721.32
2008-3         67.44
2008-4        583.00
2008-6       1524.49
2008-7      61703.90
2008-8      48289.84
2008-9      43447.50
             ...    
2016-2      97238.86
2016-3     142743.18
2016-4      86845.30
2016-5      61360.60
2016-6     100808.54
2016-7        796.66
2015-1       1297.00
2015-7        155.64
2015-9        129.00
2016-10     60183.71
2016-11     79826.98
2016-12     68761.79
2016-3      14424.88
2016-4       9826.75
2016-5       7187.81
2016-6      24921.62
2016-7      9

In [10]:
expenses = expenditures_year_month.to_frame()


In [11]:
expenses['year_month'] = expenses.index

In [12]:
expenses.columns =['amount','year_month']

In [13]:
#unique values in each column

expenses.nunique()

amount        190
year_month    128
dtype: int64

In [14]:
expenses.reset_index(inplace=True,drop=True)
#expenses

In [15]:
expenses=expenses.groupby('year_month', as_index=False).agg({"amount": "sum"})

In [16]:
expenses.nunique()

year_month    128
amount        128
dtype: int64

In [17]:
expenses.head()

,year_month,amount
0,2006-11,9066.95
1,2006-9,2085.21
2,2007-10,49288.14
3,2007-11,39262.93
4,2007-12,48468.56


In [18]:
import psycopg2 as pg
import pandas as pd
import pandas.io.sql as pd_sql

from sqlalchemy import create_engine
import psycopg2 
import io


connection_args = {
    'host': '54.200.221.80', # current aws instance public IP
    'user': 'ubuntu',    
    'dbname': 'proj3',   
    'port': 5432         
}


connection = pg.connect(**connection_args)

In [19]:
#pd.Period('2011-01')

In [20]:
cur = connection.cursor()

cur.execute("""CREATE TABLE expenses (
  year_month                TEXT,
  amount                    FLOAT
  
);""")

In [21]:
output = io.StringIO()
expenses.to_csv(output, sep='\t', header=False, index=False)
output.seek(0)
contents = output.getvalue()
cur.copy_from(output, 'expenses', null="") #null values become ''   
connection.commit()